# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, quniform
import os
import joblib

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
#Dataset created in scipt file used for training.
#I have uploaded the data to github as kaggle link requires loin to download
#you can find the dataset at the link below
#link : https://www.kaggle.com/ronitf/heart-disease-uci

ws = Workspace.from_config()
experiment_name = 'mlexphyper'

experiment=Experiment(ws, experiment_name)

## Creating Compute

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute

amlcompute_cluster_name = "cappropcompute"

# Verify that cluster does not exist already
try:
    aml_compute = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

Found existing cluster, use it.


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [5]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
ps = RandomParameterSampling({'--C': uniform(0.1, 1),
                              '--max_iter': quniform(100, 1500, 100),})

#TODO: Create your estimator and hyperdrive config
est = SKLearn("./", compute_target=aml_compute, entry_script="train.py", pip_packages=['sklearn'])

hyperdrive_run_config = HyperDriveConfig(
    estimator=est,
     hyperparameter_sampling=ps,
     policy=policy,
     primary_metric_name='Accuracy',
     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
     max_total_runs=15,
     max_concurrent_runs=4,)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [6]:
#TODO: Submit your experiment
run_hyperdrive =experiment.submit(config=hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
RunDetails(run_hyperdrive).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [9]:
best_run = run_hyperdrive.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])

['--C', '0.4142399287984764', '--max_iter', '300']


In [10]:
best_run.get_details()

{'runId': 'HD_b62c3acb-c7dc-4e7d-8d07-f61b2ea005cb_0',
 'target': 'cappropcompute',
 'status': 'Completed',
 'startTimeUtc': '2020-12-31T13:15:33.526216Z',
 'endTimeUtc': '2020-12-31T13:16:46.638724Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': 'ff031de8-14f0-4ea3-ab0b-ef8675331672',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'train.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--C', '0.4142399287984764', '--max_iter', '300'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'cappropcompute',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'jobName': None,
  'maxRunDurationSeconds': None,
  'nodeCount': 1,
  'priority': None,
  'credentialPassthrough': False,
  'environment': {'name': 'Experiment mlexphyper Environment',


### Registering and Downloading the best model

In [11]:
#TODO: Save and register the best model

model = best_run.register_model(model_name='heart-disease-hyper', model_path='outputs/heart-diesease-hyper-model.joblib')
print('Best model registered')


model.download(target_dir="outputs", exist_ok=True)
print('model downloaded')

Best model registered
model downloaded


In [12]:
# Evaluation of model 
from train import clean_data
from azureml.data.dataset_factory import TabularDatasetFactory

factory = TabularDatasetFactory()
test_data_path = "https://raw.githubusercontent.com/KrishnaB22/Azure-ML-Capstone/main/dataset/heart.csv"
test_ds = factory.from_delimited_files(test_data_path)
X_test, y_test = clean_data(test_ds)

logit_model = joblib.load('outputs/heart-diesease-hyper-model.joblib')

print(logit_model.score(X_test, y_test))

0.8712871287128713


The sklearn.linear_model.logistic module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
Trying to unpickle estimator LogisticRegression from version 0.20.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.


### The accuracy of hyperdrive model is 87%

### As the hyperdrive model has lower accuracy we do not deploy it.

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service